In [ ]:
%pip install -U upgini catboost

ERROR: Error in parse(text = input): <text>:1:1: unexpected input
1: %pip install -U upgini catboost
    ^


In [ ]:
train=df[df["date"]<"2017-01-01"]
test=df[df["date"]>="2017-01-01"]

In [ ]:
train_features=train.drop(columns=["sales"])
train_target=train["sales"]
test_features=test.drop(columns=["sales"])
test_target=test["sales"]

In [ ]:
from re import search
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

enricher=FeaturesEnricher(
    search_keys={
        "date":SearchKey.DATE,
    },
    cv=CVType.time_series
)
enricher.fit(train_features,
             train_target,
             eval_set=[(test_features, test_target)])

[============================================================] 100% Finished

<IPython.core.display.Javascript object>

WARNING #1: Search started with DATE search key only
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history

Detected task type: ModelTaskType.REGRESSION. Reason: date search key is present, treating as regression
You can set task type manually with argument `model_task_type` of FeaturesEnricher constructor if task type detected incorrectly

WARNING #2: Your training sample is unstable in number of rows per date. It is recommended to redesign the training sample



Column name,Status,Errors
target,All valid,-
date,All valid,-


<IPython.core.display.Javascript object>


Running search request, search_id=fccad836-6097-4229-a45c-ac0427555779
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
-


f_autofe_roll_2d_min_4199c37ccb,0.5437,100.0000,"120.0, 48.0, 6.0",Training dataset,AutoFE: features from Training dataset,
f_autofe_groupbythenmin_17476df384,0.2789,100.0000,"-0.0004, -0.0003, -0.0003","Upgini,Training dataset","AutoFE: feature from Markets data, grouped by feature from training dataset",Daily


Training dataset,AutoFE: features from Training dataset,0.5437,1
"Upgini,Training dataset","AutoFE: feature from Markets data, grouped by feature from training dataset",0.2789,1


Training dataset,f_autofe_roll_2d_min_4199c37ccb,target,,roll_2d_min
"Markets data, grouped by feature from training dataset",f_autofe_groupbythenmin_17476df384,f_financial_date_usd_eur_daydiff_06c94619,item_4a33ea,GroupByThenMin


In [ ]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)

enricher.calculate_metrics(
    train_features, train_target,
    eval_set=[(test_features, test_target)],
    estimator=model,
    scoring="mean_absolute_percentage_error"
)

Calculating accuracy uplift after enrichment...
y distributions from the training sample and eval_set differ according to the Kolmogorov-Smirnov test,
which makes metrics between the train and eval_set incomparable.


,Dataset type,Rows,Mean target,Enriched MAPE
0,Train,501,53.8144,0.352 ± 0.055
1,Eval 1,202,61.9109,0.275 ± 0.027


In [ ]:
enriched_train_features=enricher.transform(train_features,y=train_target,keep_input=True)
enriched_test_features=enricher.transform(test_features,y=test_target,keep_input=True)
enriched_train_features

[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 1000 rows. You have already enriched: 0 rows.
WARNING #1: Search started with DATE search key only
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history



Column name,Status,Errors
date,All valid,-




Running transform request, id=839deecd-15bc-4971-a74a-e1bad263b3d6
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
-
Retrieving selected features from data sources...


[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 1000 rows. You have already enriched: 0 rows.
WARNING #1: Search started with DATE search key only
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history



Column name,Status,Errors
date,All valid,-




Running transform request, id=48215e1f-bc7f-43a9-b797-10103d40dd35
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
-
Retrieving selected features from data sources...


,date,f_autofe_groupbythenmin_17476df384,f_autofe_roll_2d_min_4199c37ccb
0,2013-01-01,-0.00010,26.0
1,2013-01-02,-0.00014,26.0
2,2013-01-03,-0.00012,23.0
3,2013-01-05,-0.00045,12.0
4,2013-01-07,-0.00051,15.0
...,...,...,...
793,2016-12-23,-0.00045,28.0
794,2016-12-24,-0.00009,36.0
795,2016-12-26,-0.00041,57.0
796,2016-12-26,0.00000,57.0


In [ ]:
model.fit(train_features, train_target)
preds=model.predict(test_features)
eval_metric(test_target.values,preds,"SMAPE")

[41.395789274257126]

In [ ]:
model.fit(enriched_train_features, train_target)
enriched_preds=model.predict(enriched_test_features)
eval_metric(test_target.values,enriched_preds,"SMAPE")

[35.57257134638675]